In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## 필요 패키지 다운로드

In [1]:
import numpy as np      # 데이터 전처리
import pandas as pd       # 데이터 전처리
from tqdm import tqdm, tqdm_notebook    #시간 측정용
import random    # 데이터 전처리
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler # 데이터 샘플링
from keras.layers import *      # lstm_model 관련
from keras import backend as K    # lstm_model 관련
from keras import optimizers    # lstm_model 관련
import keras    # lstm_model 관련 

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


## mecab install

In [3]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 16), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [4]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [5]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 11.2MB/s 
     |████████████████████████████████| 2.3MB 64.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-01-13 02:25:22--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.2, 18.205.93.1, 18.205.93.0, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?

In [0]:
from konlpy.tag import Mecab
mecab = Mecab()

In [0]:
from konlpy.tag import Okt
okt = Okt()

## 데이터 불러오기

In [3]:
train = pd.read_csv("/content/drive/My Drive/14th/train.csv") # train.csv 불러오기
test = pd.read_csv("/content/drive/My Drive/14th/public_test.csv") # test.csv 불러오기

print(test.shape)
print(train.shape)

(1626, 3)
(295945, 4)


총 train의 행은 약 29만개, test의 행은 약 1600개로 되어있다.

In [19]:
train.head()

,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0


정상 문자와 스미싱 문자의 비율은 96:6으로 클래스 불균형 문제가 발생한다. 우리가 집중해야하는 값은 스미싱 문자이므로 적절히 undersampling 해준다.

In [6]:
smishing = len(train.loc[train['smishing']==1])
non_smishing = len(train.loc[train['smishing']==0])
num_data = len(train['smishing'])

print(round(smishing / num_data * 100, 3))
print(round(non_smishing / num_data * 100, 3))

6.32
93.68


## 데이터 전처리

### train data undersampling

소수의 클래스에 특별히 더 관심이 있는 경우에 불균형한 클래스의 균형을 맞추기 위해 undersampling으로 정상 문자와 스미싱 문자의 비율을 8:2로 조절

In [22]:
def undersampling(minor, major, train=train, random_state=47):
  X = train.drop('smishing', axis= 1)
  y= train['smishing']
  rus=RandomUnderSampler(random_state=random_state, sampling_strategy=minor/major)
  x_sampled, y_sampled = rus.fit_resample(X, y)
  col = train.columns
  train_sample = pd.DataFrame(np.hstack((x_sampled, y_sampled.reshape(-1,1))), columns=col)

  return train_sample

train_sample = undersampling(2,8)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [23]:
train_sample.shape

(93515, 4)

In [0]:
combine = [train_sample, test]
for dataset in combine:
  dataset['text(clean)'] = dataset['text'].str.replace('[,?!X-]', ' ')

# 문장에서 뜻이 될 만한 태그
p = ["NNG", "NNP", "VV", "VA", "MAG"]
# 불용어
stopwords = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ',
             'ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ']

In [0]:
def tokenizer(raw, pos=p, stopword=stopwords):

  tokened = mecab.pos(raw)

  return [word for word, tag in tokened if tag in pos and word not in stopword]


In [0]:
train_sample['tokenized'] = train_sample['text(clean)'].apply(tokenizer)
print(train_sample.shape)
train_sample.head(2)

(93515, 6)


,id,year_month,text,smishing,text(clean),tokenized
0,216114,2017-12,12월의 아름다운마무리와 성탄절 뜻깊게 보내세요.XXX용암 XXX,0,12월의 아름다운마무리와 성탄절 뜻깊게 보내세요. 용암,"[마무리, 성탄절, 뜻깊, 보내, 용암]"
1,332123,2018-12,숨가쁘게 달려운 2018년을 잘 마무리하고 2019년의 시작을 그려보는 소중한 12...,0,숨가쁘게 달려운 2018년을 잘 마무리하고 2019년의 시작을 그려보는 소중한 12...,"[숨, 가쁘, 운, 잘, 마무리, 시작, 본격, 겨울, 추위, 감기, 조심, 오늘,..."


In [0]:
# train_sample.to_csv("/content/drive/My Drive/train_stemming.csv")

In [0]:
test['tokenized'] = test['text(clean)'].apply(tokenizer)
test.head(2)

,id,year_month,text,text(clean),tokenized
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...,고객님 안녕하세요 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...,"[고객, 안녕, 새롭, 시작, 행복, 가득, 하, 기원, 지난주, 글로벌, 증시, ..."
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...,긴급 안내 은행 가락동 지점 헬리오 기본 대출이자를 ...,"[긴급, 안내, 은행, 가락동, 지점, 헬리오, 기본, 대출, 이자, 조합, 납부,..."


In [0]:
from keras.preprocessing.text import Tokenizer
# from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import Input, Model
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Concatenate, BatchNormalization, Dropout
from keras import optimizers
# from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from gensim.models import Word2Vec, Phrases
from gensim.models import KeyedVectors
import re

Using TensorFlow backend.


In [0]:
bigrams = Phrases(sentences=train_sample.tokenized)
trigrams = Phrases(sentences=bigrams[train_sample.tokenized])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
tri_model = Word2Vec(sentences = trigrams[bigrams[train_sample.tokenized]],
                     size=128, 
                     window=5, 
                     min_count=10, 
                     workers=4, 
                     sg=1)
tri_model.wv.save_word2vec_format('w2v_model') 
loaded_model = KeyedVectors.load_word2vec_format("w2v_model")

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-5433a5c69075>", line 6, in <module>
    sg=1)
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/word2vec.py", line 767, in __init__
    fast_version=FAST_VERSION)
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/base_any2vec.py", line 763, in __init__
    end_alpha=self.min_alpha, compute_loss=compute_loss)
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/word2vec.py", line 892, in train
    queue_factor=queue_factor, report_delay=report_delay, compute_loss=compute_loss, callbacks=callbacks)
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/base_any2vec.py", line 1081, in train
    **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/base_any2vec.py", line 553, in train
    total_words=total_words, queue_fa

KeyboardInterrupt: ignored

In [0]:
tri_model.wv.most_similar("광고")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('지원_팀', 0.5068578720092773),
 ('여신_영업부', 0.4908028841018677),
 ('안녕', 0.47956833243370056),
 ('신용협', 0.4731326699256897),
 ('신협_신용협', 0.4724405109882355),
 ('유진', 0.46758660674095154),
 ('듀', 0.46750351786613464),
 ('국민_평생', 0.4663026034832001),
 ('국민은행', 0.4602564871311188),
 ('신용협동조합', 0.45925480127334595)]

In [0]:
embedding_dim = 128
max_word =len(tri_model.wv.vocab)
lstm_out = 200

In [0]:
print("vocabulary size : ", len(tri_model.wv.vocab))

In [0]:
def vectorizer_data(data, vocab: dict) -> list:
  print("Vectorize sentences... ", end='\r')
  keys= list(vocab.keys())
  filter_unknown = lambda word : vocab.get(word, None) is not None
  encode = lambda review: list(map(keys.index, filter(filter_unknown, review)))
  vectorized = list(map(encode, data))
  print("Vectorize sentece... (done)")
  return vectorized

X_data = train_sample.tokenized
input_length = max(list(map(len, X_data)))
X_train = pad_sequences(
    sequences = vectorizer_data(X_data, vocab=tri_model.wv.vocab),
    maxlen= input_length,
    padding="post"
)

Vectorize sentece... (done)


In [0]:
input_length

333

In [0]:
X_test = pad_sequences(
    sequences = vectorizer_data(test.tokenized, vocab=tri_model.wv.vocab),
    maxlen= input_length,
    padding="post"
)

y_train = pd.get_dummies(train_sample['smishing']).values

Vectorize sentece... (done)


In [0]:
def matthews_correlation(y_true, y_pred):
  '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
  y_pred_pos = K.round(K.clip(y_pred, 0, 1))
  y_pred_neg = 1 - y_pred_pos

  y_pos = K.round(K.clip(y_true, 0, 1))
  y_neg = 1 - y_pos

  tp = K.sum(y_pos * y_pred_pos)
  tn = K.sum(y_neg * y_pred_neg)

  fp = K.sum(y_neg * y_pred_pos)
  fn = K.sum(y_pos * y_pred_neg)

  numerator = (tp * tn - fp * fn)
  denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

  return numerator / (denominator + K.epsilon())

가중치 초기값으로 Xavier 초기값과 He 초기값중 He 초기값의 성능이 더 좋았음.

In [0]:
class Attention(Layer):

  def __init__(self, step_dim,
                W_regularizer=None, b_regularizer=None,
                W_constraint=None, b_constraint=None,
                bias=True, **kwargs):
      self.supports_masking = True
      # self.init = keras.initializers.get('glorot_uniform')
      self.init = keras.initializers.he_uniform
      self.W_regularizer = keras.regularizers.get(W_regularizer)
      self.b_regularizer = keras.regularizers.get(b_regularizer)

      self.W_constraint = keras.constraints.get(W_constraint)
      self.b_constraint = keras.constraints.get(b_constraint)

      self.bias = bias
      self.step_dim = step_dim
      self.features_dim = 0
      super(Attention, self).__init__(**kwargs)

  def build(self, input_shape):
      assert len(input_shape) == 3

      self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
      self.features_dim = input_shape[-1]

      if self.bias:
          self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
      else:
          self.b = None

      self.built = True

  def compute_mask(self, input, input_mask=None):
      return None

  def call(self, x, mask=None):
      features_dim = self.features_dim
      step_dim = self.step_dim

      eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

      if self.bias:
          eij += self.b

      eij = K.tanh(eij)

      a = K.exp(eij)

      if mask is not None:
          a *= K.cast(mask, K.floatx())

      a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

      a = K.expand_dims(a)
      weighted_input = x * a
      return K.sum(weighted_input, axis=1)

  def compute_output_shape(self, input_shape):
      return input_shape[0],  self.features_dim

## model(2den)

In [0]:
def build_model(embedding_matrix:np.ndarray, input_length: int):
  model = keras.Sequential()
  model.add(Embedding(
      input_dim = embedding_matrix.shape[0],
      output_dim = embedding_matrix.shape[1], 
      input_length = input_length,
      weights = [embedding_matrix],
      trainable=False
  ))
  model.add(Bidirectional(LSTM(128,dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
  model.add(Attention(input_length))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(2, activation="softmax"))
  adam = optimizers.Adam(lr=0.005, amsgrad = True)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  model.summary()
  return model

model_lstm = build_model(embedding_matrix=tri_model.wv.vectors, input_length=X_train.shape[1])
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                        min_delta=0,
                                                        patience=1,
                                                        verbose=0, mode='auto')

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 128)          1355520   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 300, 256)          263168    
_________________________________________________________________
attention_3 (Attention)      (None, 256)               556       
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 514       
Total params: 1,619,758
Trainable params: 264,238
Non-trainable params: 1,355,520
_________________________________________________________________


In [0]:
batch_size = 128
# epo 10 + 1
model_lstm.fit(X_train, y_train, epochs=2, batch_size=batch_size, validation_split = 3, callbacks=[early_stopping_callback])

# model2(1den)

In [0]:
def build_model(embedding_matrix:np.ndarray, input_length: int):
  model = keras.Sequential()
  model.add(Embedding(
      input_dim = embedding_matrix.shape[0],
      output_dim = embedding_matrix.shape[1], 
      input_length = input_length,
      weights = [embedding_matrix],
      trainable=False
  ))
  model.add(Bidirectional(LSTM(128,dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
  model.add(Attention(input_length))
  # model.add(Dropout(0.5))
  # model.add(Dense(32, activation='relu'))
  # model.add(Dropout(0.4))
  model.add(Dense(2, activation="softmax"))
  adam = optimizers.Adam(lr=0.005, amsgrad = True)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  model.summary()
  return model

model_lstm = build_model(embedding_matrix=tri_model.wv.vectors, input_length=X_train.shape[1])
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                        min_delta=0,
                                                        patience=1,
                                                        verbose=0, mode='auto')

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 300, 128)          1355520   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 300, 256)          263168    
_________________________________________________________________
attention_5 (Attention)      (None, 256)               556       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 514       
Total params: 1,619,758
Trainable params: 264,238
Non-trainable params: 1,355,520
_________________________________________________________________


In [0]:
batch_size = 128
# epo 6 + 1
model_lstm.fit(X_train, y_train, epochs=1, batch_size=batch_size, validation_split = 3, callbacks=[early_stopping_callback])

Epoch 1/1
93515/93515 [==============================] - 846s 9ms/step - loss: 0.0012 - acc: 0.9998


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


# model3(epo256)

In [0]:
def build_model(embedding_matrix:np.ndarray, input_length: int):
  model = keras.Sequential()
  model.add(Embedding(
      input_dim = embedding_matrix.shape[0],
      output_dim = embedding_matrix.shape[1], 
      input_length = input_length,
      weights = [embedding_matrix],
      trainable=False
  ))
  model.add(Bidirectional(LSTM(128,dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
  model.add(Attention(input_length))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(2, activation="softmax"))
  adam = optimizers.Adam(lr=0.005, amsgrad = True)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  model.summary()
  return model

model_lstm = build_model(embedding_matrix=tri_model.wv.vectors, input_length=X_train.shape[1])
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                        min_delta=0,
                                                        patience=1,
                                                        verbose=0, mode='auto')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 300, 128)          1355520   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 300, 256)          263168    
_________________________________________________________________
attention_2 (Attention)      (None, 256)               556       
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                

In [0]:
batch_size = 256
# epo 10 + 2 + 3
model_lstm.fit(X_train, y_train, epochs=3, batch_size=batch_size, validation_split = 3, callbacks=[early_stopping_callback])

Epoch 1/3
93515/93515 [==============================] - 321s 3ms/step - loss: 0.0019 - acc: 0.9996
Epoch 2/3


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


93515/93515 [==============================] - 317s 3ms/step - loss: 0.0021 - acc: 0.9996
Epoch 3/3
93515/93515 [==============================] - 312s 3ms/step - loss: 0.0023 - acc: 0.9996


# model4(lr=0.008)

In [0]:
def build_model(embedding_matrix:np.ndarray, input_length: int):
  model = keras.Sequential()
  model.add(Embedding(
      input_dim = embedding_matrix.shape[0],
      output_dim = embedding_matrix.shape[1], 
      input_length = input_length,
      weights = [embedding_matrix],
      trainable=False
  ))
  model.add(Bidirectional(LSTM(128,dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
  model.add(Attention(input_length))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(2, activation="softmax"))
  adam = optimizers.Adam(lr=0.008, amsgrad = True)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  model.summary()
  return model

model_lstm = build_model(embedding_matrix=tri_model.wv.vectors, input_length=X_train.shape[1])
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                        min_delta=0,
                                                        patience=1,
                                                        verbose=0, mode='auto')

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 128)          1355520   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 300, 256)          263168    
_________________________________________________________________
attention_3 (Attention)      (None, 256)               556       
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                

In [0]:
batch_size = 256
# epo 20
model_lstm.fit(X_train, y_train, epochs=20, batch_size=batch_size, validation_split = 3, callbacks=[early_stopping_callback])

Epoch 1/20
93515/93515 [==============================] - 318s 3ms/step - loss: 0.0325 - acc: 0.9892
Epoch 2/20


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


93515/93515 [==============================] - 316s 3ms/step - loss: 0.0114 - acc: 0.9974
Epoch 3/20
93515/93515 [==============================] - 314s 3ms/step - loss: 0.0078 - acc: 0.9983
Epoch 4/20
93515/93515 [==============================] - 314s 3ms/step - loss: 0.0048 - acc: 0.9990
Epoch 5/20
93515/93515 [==============================] - 316s 3ms/step - loss: 0.0042 - acc: 0.9991
Epoch 6/20
93515/93515 [==============================] - 322s 3ms/step - loss: 0.0038 - acc: 0.9990
Epoch 7/20
93515/93515 [==============================] - 330s 4ms/step - loss: 0.0031 - acc: 0.9994
Epoch 8/20
93515/93515 [==============================] - 325s 3ms/step - loss: 0.0027 - acc: 0.9995
Epoch 9/20
93515/93515 [==============================] - 327s 3ms/step - loss: 0.0027 - acc: 0.9995
Epoch 10/20
93515/93515 [==============================] - 327s 4ms/step - loss: 0.0031 - acc: 0.9994
Epoch 11/20
93515/93515 [==============================] - 329s 4ms/step - loss: 0.0023 - acc: 0.9996

# model5(den3,input_length=333, lr=0.003)

In [0]:
def build_model(embedding_matrix:np.ndarray, input_length: int):
  model = keras.Sequential()
  model.add(Embedding(
      input_dim = embedding_matrix.shape[0],
      output_dim = embedding_matrix.shape[1], 
      input_length = input_length,
      weights = [embedding_matrix],
      trainable=False
  ))
  model.add(Bidirectional(LSTM(128,dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
  model.add(Attention(input_length))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(2, activation="softmax"))
  adam = optimizers.Adam(lr=0.003, amsgrad = True)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  model.summary()
  return model

model_lstm = build_model(embedding_matrix=tri_model.wv.vectors, input_length=X_train.shape[1])
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                        min_delta=0,
                                                        patience=1,
                                                        verbose=0, mode='auto')











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 333, 128)          1355520   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 333, 256)          263168    
_________________________________________________________________
attention_1 (Attention)      (None, 256)               589       
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0   

In [0]:
batch_size = 512
# epo 10
model_lstm.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_split = 3, callbacks=[early_stopping_callback])

Epoch 1/10
93515/93515 [==============================] - 1713s 18ms/step - loss: 0.0196 - acc: 0.9956
Epoch 2/10


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


93515/93515 [==============================] - 1695s 18ms/step - loss: 0.0100 - acc: 0.9979
Epoch 3/10
93515/93515 [==============================] - 1690s 18ms/step - loss: 0.0067 - acc: 0.9984
Epoch 4/10
84992/93515 [==========================>...] - ETA: 2:34 - loss: 0.0043 - acc: 0.9991

# model6(init=he)- 오빠 계정

In [0]:
class Attention1(Layer):

  def __init__(self, step_dim,
                W_regularizer=None, b_regularizer=None,
                W_constraint=None, b_constraint=None,
                bias=True, **kwargs):
      self.supports_masking = True
      self.init = keras.initializers.he_normal(seed=47)

      self.W_regularizer = keras.regularizers.get(W_regularizer)
      self.b_regularizer = keras.regularizers.get(b_regularizer)

      self.W_constraint = keras.constraints.get(W_constraint)
      self.b_constraint = keras.constraints.get(b_constraint)

      self.bias = bias
      self.step_dim = step_dim
      self.features_dim = 0
      super(Attention, self).__init__(**kwargs)

  def build(self, input_shape):
      assert len(input_shape) == 3

      self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
      self.features_dim = input_shape[-1]

      if self.bias:
          self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
      else:
          self.b = None

      self.built = True

  def compute_mask(self, input, input_mask=None):
      return None

  def call(self, x, mask=None):
      features_dim = self.features_dim
      step_dim = self.step_dim

      eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

      if self.bias:
          eij += self.b

      eij = K.tanh(eij)

      a = K.exp(eij)

      if mask is not None:
          a *= K.cast(mask, K.floatx())

      a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

      a = K.expand_dims(a)
      weighted_input = x * a
      return K.sum(weighted_input, axis=1)

  def compute_output_shape(self, input_shape):
      return input_shape[0],  self.features_dim

In [0]:
def build_model(embedding_matrix:np.ndarray, input_length: int):
  model = keras.Sequential()
  model.add(Embedding(
      input_dim = embedding_matrix.shape[0],
      output_dim = embedding_matrix.shape[1], 
      input_length = input_length,
      weights = [embedding_matrix],
      trainable=False
  ))
  model.add(Bidirectional(LSTM(128,dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
  model.add(Attention(input_length))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(2, activation="softmax"))
  adam = optimizers.Adam(lr=0.003, amsgrad = True)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  model.summary()
  return model

model_lstm = build_model(embedding_matrix=tri_model.wv.vectors, input_length=X_train.shape[1])
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                        min_delta=0,
                                                        patience=1,
                                                        verbose=0, mode='auto')

# model7(re_dropout=0.4), init=he - 언니 계정

In [0]:
class Attention(Layer):

  def __init__(self, step_dim,
                W_regularizer=None, b_regularizer=None,
                W_constraint=None, b_constraint=None,
                bias=True, **kwargs):
      self.supports_masking = True
      self.init = keras.initializers.he_normal(seed=47)

      self.W_regularizer = keras.regularizers.get(W_regularizer)
      self.b_regularizer = keras.regularizers.get(b_regularizer)

      self.W_constraint = keras.constraints.get(W_constraint)
      self.b_constraint = keras.constraints.get(b_constraint)

      self.bias = bias
      self.step_dim = step_dim
      self.features_dim = 0
      super(Attention, self).__init__(**kwargs)

  def build(self, input_shape):
      assert len(input_shape) == 3

      self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
      self.features_dim = input_shape[-1]

      if self.bias:
          self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
      else:
          self.b = None

      self.built = True

  def compute_mask(self, input, input_mask=None):
      return None

  def call(self, x, mask=None):
      features_dim = self.features_dim
      step_dim = self.step_dim

      eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

      if self.bias:
          eij += self.b

      eij = K.tanh(eij)

      a = K.exp(eij)

      if mask is not None:
          a *= K.cast(mask, K.floatx())

      a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

      a = K.expand_dims(a)
      weighted_input = x * a
      return K.sum(weighted_input, axis=1)

  def compute_output_shape(self, input_shape):
      return input_shape[0],  self.features_dim

In [0]:
def build_model(embedding_matrix:np.ndarray, input_length: int):
  model = keras.Sequential()
  model.add(Embedding(
      input_dim = embedding_matrix.shape[0],
      output_dim = embedding_matrix.shape[1], 
      input_length = input_length,
      weights = [embedding_matrix],
      trainable=False
  ))
  model.add(Bidirectional(LSTM(128,dropout=0.4, recurrent_dropout=0.4,return_sequences=True)))
  model.add(Attention(input_length))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(2, activation="softmax"))
  adam = optimizers.Adam(lr=0.003, amsgrad = True)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  model.summary()
  return model

model_lstm = build_model(embedding_matrix=tri_model.wv.vectors, input_length=X_train.shape[1])
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                        min_delta=0,
                                                        patience=1,
                                                        verbose=0, mode='auto')

In [0]:
batch_size = 256
# epo 10
model_lstm.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_split = 3, callbacks=[early_stopping_callback])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
93515/93515 [==============================] - 509s 5ms/step - loss: 0.0445 - acc: 0.9844
Epoch 2/10


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


93515/93515 [==============================] - 493s 5ms/step - loss: 0.0122 - acc: 0.9973
Epoch 3/10
93515/93515 [==============================] - 488s 5ms/step - loss: 0.0081 - acc: 0.9979
Epoch 4/10
93515/93515 [==============================] - 495s 5ms/step - loss: 0.0066 - acc: 0.9984
Epoch 5/10
93515/93515 [==============================] - 491s 5ms/step - loss: 0.0053 - acc: 0.9989
Epoch 6/10
93515/93515 [==============================] - 491s 5ms/step - loss: 0.0045 - acc: 0.9990
Epoch 7/10
93515/93515 [==============================] - 490s 5ms/step - loss: 0.0043 - acc: 0.9990
Epoch 8/10
93515/93515 [==============================] - 491s 5ms/step - loss: 0.0034 - acc: 0.9994
Epoch 9/10
93515/93515 [==============================] - 486s 5ms/step - loss: 0.0034 - acc: 0.9993
Epoch 10/10
93515/93515 [==============================] - 488s 5ms/step - loss: 0.0031 - acc: 0.9993


# model8(binary-cr)

In [0]:
def build_model(embedding_matrix:np.ndarray, input_length: int):
  model = keras.Sequential()
  model.add(Embedding(
      input_dim = embedding_matrix.shape[0],
      output_dim = embedding_matrix.shape[1], 
      input_length = input_length,
      weights = [embedding_matrix],
      trainable=False
  ))
  model.add(Bidirectional(LSTM(128,dropout=0.4, recurrent_dropout=0.4,return_sequences=True)))
  model.add(Attention(input_length))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(2, activation="softmax"))
  adam = optimizers.Adam(lr=0.003, amsgrad = True)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
  model.summary()
  return model

model_lstm = build_model(embedding_matrix=tri_model.wv.vectors, input_length=X_train.shape[1])
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                        min_delta=0,
                                                        patience=1,
                                                        verbose=0, mode='auto')

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 333, 128)          1355520   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 333, 256)          263168    
_________________________________________________________________
attention_2 (Attention)      (None, 256)               589       
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)               

In [0]:
batch_size = 256
# epo 10
model_lstm.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_split = 3, callbacks=[early_stopping_callback])

Epoch 1/10
93515/93515 [==============================] - 500s 5ms/step - loss: 0.0497 - acc: 0.9824
Epoch 2/10


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


93515/93515 [==============================] - 495s 5ms/step - loss: 0.0136 - acc: 0.9967
Epoch 3/10
93515/93515 [==============================] - 500s 5ms/step - loss: 0.0086 - acc: 0.9979
Epoch 4/10
93515/93515 [==============================] - 500s 5ms/step - loss: 0.0070 - acc: 0.9984
Epoch 5/10
93515/93515 [==============================] - 501s 5ms/step - loss: 0.0057 - acc: 0.9987
Epoch 6/10
93515/93515 [==============================] - 497s 5ms/step - loss: 0.0057 - acc: 0.9987
Epoch 7/10
93515/93515 [==============================] - 504s 5ms/step - loss: 0.0046 - acc: 0.9989
Epoch 8/10
93515/93515 [==============================] - 504s 5ms/step - loss: 0.0039 - acc: 0.9990
Epoch 9/10
93515/93515 [==============================] - 496s 5ms/step - loss: 0.0046 - acc: 0.9990
Epoch 10/10
93515/93515 [==============================] - 496s 5ms/step - loss: 0.0040 - acc: 0.9991


# model10(=model9 large dropout)

In [0]:
tri_model = Word2Vec(sentences = trigrams[bigrams[train_sample.tokenized]],
                     size=256, 
                     window=5, 
                     min_count=10, 
                     workers=4, 
                     sg=1)
tri_model.wv.save_word2vec_format('w2v_model') 
loaded_model = KeyedVectors.load_word2vec_format("w2v_model")

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
embedding_dim = 256
max_word =len(tri_model.wv.vocab)
lstm_out = 200

In [0]:
def vectorizer_data(data, vocab: dict) -> list:
  print("Vectorize sentences... ", end='\r')
  keys= list(vocab.keys())
  filter_unknown = lambda word : vocab.get(word, None) is not None
  encode = lambda review: list(map(keys.index, filter(filter_unknown, review)))
  vectorized = list(map(encode, data))
  print("Vectorize sentece... (done)")
  return vectorized

X_data = train_sample.tokenized
input_length = max(list(map(len, X_data)))
X_train = pad_sequences(
    sequences = vectorizer_data(X_data, vocab=tri_model.wv.vocab),
    maxlen= input_length,
    padding="post"
)

Vectorize sentece... (done)


In [0]:
X_test = pad_sequences(
    sequences = vectorizer_data(test.tokenized, vocab=tri_model.wv.vocab),
    maxlen= input_length,
    padding="post"
)

y_train = pd.get_dummies(train_sample['smishing']).values

Vectorize sentece... (done)


In [0]:
def build_model(embedding_matrix:np.ndarray, input_length: int):
  model = keras.Sequential()
  model.add(Embedding(
      input_dim = embedding_matrix.shape[0],
      output_dim = embedding_matrix.shape[1], 
      input_length = input_length,
      weights = [embedding_matrix],
      trainable=False
  ))
  model.add(Bidirectional(LSTM(128,dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
  model.add(Attention(input_length))
  model.add(Dropout(0.7))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.6))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(2, activation="softmax"))
  adam = optimizers.Adam(lr=0.003, amsgrad = True)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  model.summary()
  return model

model_lstm = build_model(embedding_matrix=tri_model.wv.vectors, input_length=X_train.shape[1])
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                        min_delta=0,
                                                        patience=1,
                                                        verbose=0, mode='auto')

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 333, 256)          2711040   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 333, 256)          394240    
_________________________________________________________________
attention_4 (Attention)      (None, 256)               589       
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                16448     
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 32)               

In [0]:
batch_size = 512
# epo 10 +7+5
model_lstm.fit(X_train, y_train, epochs=5, batch_size=batch_size, validation_split = 3, callbacks=[early_stopping_callback])

Epoch 1/5
93515/93515 [==============================] - 246s 3ms/step - loss: 0.0022 - acc: 0.9997
Epoch 2/5


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


93515/93515 [==============================] - 246s 3ms/step - loss: 0.0024 - acc: 0.9996
Epoch 3/5
93515/93515 [==============================] - 246s 3ms/step - loss: 0.0024 - acc: 0.9997
Epoch 4/5
93515/93515 [==============================] - 248s 3ms/step - loss: 0.0018 - acc: 0.9998
Epoch 5/5
93515/93515 [==============================] - 248s 3ms/step - loss: 0.0020 - acc: 0.9997


# model13(=model9 less dropout,binary) 0.949327

In [0]:
class Attention1(Layer):

  def __init__(self, step_dim,
                W_regularizer=None, b_regularizer=None,
                W_constraint=None, b_constraint=None,
                bias=True, **kwargs):
      self.supports_masking = True
      self.init = keras.initializers.he_normal(seed=47)

      self.W_regularizer = keras.regularizers.get(W_regularizer)
      self.b_regularizer = keras.regularizers.get(b_regularizer)

      self.W_constraint = keras.constraints.get(W_constraint)
      self.b_constraint = keras.constraints.get(b_constraint)

      self.bias = bias
      self.step_dim = step_dim
      self.features_dim = 0
      super(Attention, self).__init__(**kwargs)

  def build(self, input_shape):
      assert len(input_shape) == 3

      self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
      self.features_dim = input_shape[-1]

      if self.bias:
          self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
      else:
          self.b = None

      self.built = True

  def compute_mask(self, input, input_mask=None):
      return None

  def call(self, x, mask=None):
      features_dim = self.features_dim
      step_dim = self.step_dim

      eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

      if self.bias:
          eij += self.b

      eij = K.tanh(eij)

      a = K.exp(eij)

      if mask is not None:
          a *= K.cast(mask, K.floatx())

      a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

      a = K.expand_dims(a)
      weighted_input = x * a
      return K.sum(weighted_input, axis=1)

  def compute_output_shape(self, input_shape):
      return input_shape[0],  self.features_dim

In [0]:
def build_model(embedding_matrix:np.ndarray, input_length: int):
  model = keras.Sequential()
  model.add(Embedding(
      input_dim = embedding_matrix.shape[0],
      output_dim = embedding_matrix.shape[1], 
      input_length = input_length,
      weights = [embedding_matrix],
      trainable=False
  ))
  model.add(Bidirectional(LSTM(128,dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
  model.add(Attention(input_length))
  model.add(Dropout(0.3))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(2, activation="softmax"))
  adam = optimizers.Adam(lr=0.003, amsgrad = True)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
  model.summary()
  return model

model_lstm = build_model(embedding_matrix=tri_model.wv.vectors, input_length=X_train.shape[1])
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                        min_delta=0,
                                                        patience=1,
                                                        verbose=0, mode='auto')

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 333, 256)          2711040   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 333, 256)          394240    
_________________________________________________________________
attention_7 (Attention)      (None, 256)               589       
_________________________________________________________________
dropout_19 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 64)                16448     
_________________________________________________________________
dropout_20 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 32)               

In [0]:
batch_size = 512
# epo 20+3
model_lstm.fit(X_train, y_train, epochs=21, batch_size=batch_size, validation_split = 3, callbacks=[early_stopping_callback])

Epoch 1/21
93515/93515 [==============================] - 265s 3ms/step - loss: 0.0384 - acc: 0.9869
Epoch 2/21


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


93515/93515 [==============================] - 269s 3ms/step - loss: 0.0086 - acc: 0.9979
Epoch 3/21
93515/93515 [==============================] - 268s 3ms/step - loss: 0.0056 - acc: 0.9987
Epoch 4/21
93515/93515 [==============================] - 266s 3ms/step - loss: 0.0039 - acc: 0.9992
Epoch 5/21
93515/93515 [==============================] - 270s 3ms/step - loss: 0.0037 - acc: 0.9992
Epoch 6/21
93515/93515 [==============================] - 264s 3ms/step - loss: 0.0026 - acc: 0.9994
Epoch 7/21
93515/93515 [==============================] - 260s 3ms/step - loss: 0.0025 - acc: 0.9994
Epoch 8/21
93515/93515 [==============================] - 267s 3ms/step - loss: 0.0018 - acc: 0.9996
Epoch 9/21
93515/93515 [==============================] - 264s 3ms/step - loss: 0.0017 - acc: 0.9997
Epoch 10/21
93515/93515 [==============================] - 258s 3ms/step - loss: 0.0019 - acc: 0.9996
Epoch 11/21
93515/93515 [==============================] - 257s 3ms/step - loss: 0.0013 - acc: 0.9998

# model14(=model9 + low lr)

In [0]:
def build_model(embedding_matrix:np.ndarray, input_length: int):
  model = keras.Sequential()
  model.add(Embedding(
      input_dim = embedding_matrix.shape[0],
      output_dim = embedding_matrix.shape[1], 
      input_length = input_length,
      weights = [embedding_matrix],
      trainable=False
  ))
  model.add(Bidirectional(LSTM(128,dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
  model.add(Attention(input_length))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(2, activation="softmax"))
  adam = optimizers.Adam(lr=0.0009, amsgrad = True)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  model.summary()
  return model

model_lstm = build_model(embedding_matrix=tri_model.wv.vectors, input_length=X_train.shape[1])
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                        min_delta=0,
                                                        patience=1,
                                                        verbose=0, mode='auto')

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 333, 256)          2711040   
_________________________________________________________________
bidirectional_9 (Bidirection (None, 333, 256)          394240    
_________________________________________________________________
attention_8 (Attention)      (None, 256)               589       
_________________________________________________________________
dropout_22 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 64)                16448     
_________________________________________________________________
dropout_23 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 32)               

In [0]:
batch_size = 512
# epo 20
model_lstm.fit(X_train, y_train, epochs=20, batch_size=batch_size, validation_split = 3, callbacks=[early_stopping_callback])

Epoch 1/20
93515/93515 [==============================] - 258s 3ms/step - loss: 0.0807 - acc: 0.9676
Epoch 2/20


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


93515/93515 [==============================] - 257s 3ms/step - loss: 0.0160 - acc: 0.9967
Epoch 3/20
93515/93515 [==============================] - 260s 3ms/step - loss: 0.0110 - acc: 0.9975
Epoch 4/20
93515/93515 [==============================] - 257s 3ms/step - loss: 0.0079 - acc: 0.9982
Epoch 5/20
93515/93515 [==============================] - 258s 3ms/step - loss: 0.0072 - acc: 0.9985
Epoch 6/20
93515/93515 [==============================] - 254s 3ms/step - loss: 0.0056 - acc: 0.9988
Epoch 7/20
93515/93515 [==============================] - 252s 3ms/step - loss: 0.0048 - acc: 0.9990
Epoch 8/20
93515/93515 [==============================] - 251s 3ms/step - loss: 0.0038 - acc: 0.9993
Epoch 9/20
93515/93515 [==============================] - 258s 3ms/step - loss: 0.0031 - acc: 0.9995
Epoch 10/20
93515/93515 [==============================] - 263s 3ms/step - loss: 0.0030 - acc: 0.9993
Epoch 11/20
93515/93515 [==============================] - 262s 3ms/step - loss: 0.0029 - acc: 0.9993

# model15(embd)

In [0]:
tri_model = Word2Vec(sentences = trigrams[bigrams[train_sample.tokenized]],
                     size=256, 
                     window=10, 
                     min_count=10, 
                     workers=4, 
                     sg=1)
tri_model.wv.save_word2vec_format('w2v_model') 
loaded_model = KeyedVectors.load_word2vec_format("w2v_model")

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def vectorizer_data(data, vocab: dict) -> list:
  print("Vectorize sentences... ", end='\r')
  keys= list(vocab.keys())
  filter_unknown = lambda word : vocab.get(word, None) is not None
  encode = lambda review: list(map(keys.index, filter(filter_unknown, review)))
  vectorized = list(map(encode, data))
  print("Vectorize sentece... (done)")
  return vectorized

X_data = train_sample.tokenized
input_length = max(list(map(len, X_data)))
X_train = pad_sequences(
    sequences = vectorizer_data(X_data, vocab=tri_model.wv.vocab),
    maxlen= input_length,
    padding="post"
)

Vectorize sentece... (done)


In [0]:
X_test = pad_sequences(
    sequences = vectorizer_data(test.tokenized, vocab=tri_model.wv.vocab),
    maxlen= input_length,
    padding="post"
)

y_train = pd.get_dummies(train_sample['smishing']).values

Vectorize sentece... (done)


In [0]:
def build_model(embedding_matrix:np.ndarray, input_length: int):
  model = keras.Sequential()
  model.add(Embedding(
      input_dim = embedding_matrix.shape[0],
      output_dim = embedding_matrix.shape[1], 
      input_length = input_length,
      weights = [embedding_matrix],
      trainable=False
  ))
  model.add(Bidirectional(LSTM(256,dropout=0.4, recurrent_dropout=0.4,return_sequences=True)))
  model.add(Attention(input_length))
  model.add(Dropout(0.5))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(2, activation="softmax"))
  adam = optimizers.Adam(lr=0.005, amsgrad = True)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  model.summary()
  return model

model_lstm = build_model(embedding_matrix=tri_model.wv.vectors, input_length=X_train.shape[1])
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                        min_delta=0,
                                                        patience=1,
                                                        verbose=0, mode='auto')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 333, 256)          2711040   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 333, 512)          1050624   
_________________________________________________________________
attention_2 (Attention)      (None, 512)               845       
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)               

In [0]:
batch_size = 600
# epo 20
model_lstm.fit(X_train, y_train, epochs=5, batch_size=batch_size, validation_split = 3, callbacks=[early_stopping_callback])

Epoch 1/5
93515/93515 [==============================] - 197s 2ms/step - loss: 0.0030 - acc: 0.9994
Epoch 2/5


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


93515/93515 [==============================] - 198s 2ms/step - loss: 0.0026 - acc: 0.9994
Epoch 3/5
93515/93515 [==============================] - 198s 2ms/step - loss: 0.0029 - acc: 0.9993
Epoch 4/5
93515/93515 [==============================] - 199s 2ms/step - loss: 0.0024 - acc: 0.9996
Epoch 5/5
93515/93515 [==============================] - 200s 2ms/step - loss: 0.0025 - acc: 0.9996


In [0]:
prediction = model_lstm.predict(X_test)
proba = model_lstm.predict_proba(X_test)

smishing_proba=list(map(lambda x: x[1],proba))

submission = pd.read_csv('/content/drive/My Drive/submission.csv', index_col= 'id')
submission['smishing'] = smishing_proba
submission.to_csv("/content/drive/My Drive/submission.csv")

In [0]:
prediction[:10]

array([[9.9999988e-01, 8.0860850e-08],
       [9.9999738e-01, 2.6521898e-06],
       [1.0000000e+00, 5.6414216e-09],
       [9.9999940e-01, 5.7474483e-07],
       [1.0000000e+00, 1.1674307e-09],
       [9.9999976e-01, 2.6718112e-07],
       [9.9999976e-01, 2.1076930e-07],
       [9.9999416e-01, 5.8812825e-06],
       [1.0000000e+00, 9.9593391e-09],
       [1.0000000e+00, 1.3367470e-08]], dtype=float32)

In [0]:
submission.head(20)

,smishing
id,
340000,1.977901e-06
340001,7.386830e-06
340002,9.556693e-08
340003,1.670602e-07
340004,2.092865e-07
340005,8.233961e-07
340006,9.213698e-08
340007,1.223019e-06
340008,4.203817e-07


# end

In [0]:
model.wv.vectors.shape

(6086, 128)

In [0]:
print(X_train.shape)
print(X_test.shape)